## 데이터로드

In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

import pandas as pd

import os
import re

from dotenv import load_dotenv
load_dotenv()

def get_file_names(folder_path, format=".pdf"):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.
    """
    import os
    
    try:
        all_files = os.listdir(folder_path)
        pdf_files = [file.replace(format,"") for file in all_files if file.lower().endswith(format)]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []
    

In [2]:
def load_and_split_tax_law(file_name):
    
    """
    PDF 파일들을 처리하여 임베딩을 Chroma Vector Store에 저장합니다.
    """

    file_path = f"data/tax_law/{file_name}.pdf"

    loader = PyMuPDFLoader(file_path)
    load_document = loader.load()

    # 전처리 - 반복 텍스트 삭제
    delete_pattern_1 = rf"법제처\s*\d+\s*국가법령정보센터\n{file_name.replace('_', ' ')}\n" 
    delete_pattern_2 = r'\[[\s\S]*?\]'
    delete_pattern_3 = r'<[\s\S]*?>'
    
    full_text = " ".join([document.page_content for document in load_document])
    
    full_text = re.sub(delete_pattern_1, "", full_text)
    full_text=re.sub(delete_pattern_2, '', full_text)
    full_text=re.sub(delete_pattern_3, '', full_text)
    
    # 전처리 - split
    split_pattern = r"\s*\n(제\d+조(?:의\d+)?(?:\([^)]*\))?)(?=\s|$)"
    chunks = re.split(split_pattern, full_text)

    chunk_docs = []  # 최종 return 할 list
    connected_chunks = [] 
    current_chunk = ""
    is_buchik_section = False 
    
    # 전처리 - 일반 조항과 부칙의 조항과 구별하기 위해 접두어 '부칙-'을 넣음.
    for chunk in chunks:
        if re.search(r"\n\s*부칙", chunk): 
            is_buchik_section = True

        if chunk.startswith("제") and "조" in chunk: 
            if is_buchik_section:
                chunk = "부칙-" + chunk

            if current_chunk:
                connected_chunks.append(current_chunk.strip())
            current_chunk = chunk 
        else:
            current_chunk += f" {chunk}" 

    if current_chunk:
        connected_chunks.append(current_chunk.strip())

    for chunk in connected_chunks:
        pattern =  r"^(?:부칙-)?제\d+조(?:의\d*)?(?:\([^)]*\))?"

        keyword = ''
        keyword += file_name
        
        match = re.search(pattern, chunk)
        if match:
            word = match.group()
            word = re.sub(r'\s+', ' ', word)
            keyword += word 
            
        doc = Document(metadata={"title": file_name, "keyword":keyword, "effective_year": 2025 }, page_content=chunk),
        
        chunk_docs.extend(doc)
        
    return chunk_docs

In [3]:
from load_functions import (
    load_2024핵심개정세법, 
    load_연말정산신고안내,
    load_주택자금공제의이해, 
    load_주요공제계산사례
)


# vector store 에 넣을 데이터 document 모으기 -> all_documents
all_documents = []
# load - 세법 
law_files = get_file_names("data/tax_law")
for file in law_files:
    all_documents.extend(load_and_split_tax_law(file))
    
# load - 2024_핵심_개정세법
all_documents.extend(load_2024핵심개정세법())
# load - 연말정산_신고안내
all_documents.extend(load_연말정산신고안내())
# load - 주택자금공제의이해
all_documents.extend(load_주택자금공제의이해())
# load - 주요공제계산사례
all_documents.extend(load_주요공제계산사례())


c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 0 of document data/tax_etc/2024_핵심_개정세법.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 1 of document data/tax_etc/2024_핵심_개정세법.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 15 of document data/tax_etc/2024_핵심_개정세법.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 17 of document data/tax_etc/2024_핵심_개정세법.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\par

In [15]:
# len(load_연말정산신고안내()) - 318
# len(load_주요공제계산사례()) - 30
# len(load_2024핵심개정세법())  - 60
# len(load_주택자금공제의이해()) - 60
5431 + 318 + 30 + 60 + 60

5899

In [4]:
len(all_documents)

5899

In [5]:
COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

def set_vector_store(documents):
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

    return Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )

In [6]:
# 📌 vector store 생성
# vector_store = set_vector_store(all_documents)


In [7]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 10}
)

In [17]:
# Prompt Template 생성
messages = [
        ("ai", """
        당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 저장된 세법 조항 데이터를 기반으로 사용자 질문에 답변하세요.

        - 모든 답변은 학습된 세법 데이터 내에서만 유효한 정보를 바탕으로 작성하세요. 데이터에 없는 내용은 추측하거나 임의로 생성하지 마세요.
        - 질문에 명확한 답변이 없거나 데이터 내에서 찾을 수 없는 경우, 정직하게 "잘 모르겠습니다."라고 말하고, 새로운 질문을 유도하세요.
        - 질문이 포함된 조항뿐 아니라, 필요 시 서로 연관된 다른 조항도 참고하여 답변의 정확성과 완성도를 높이세요.
        - 사용자가 이해하기 쉽게 답변을 구성하며, 중요한 키워드나 법 조항은 명확히 표시하세요.
        - 세법과 관련된 복잡한 질문에 대해서는 관련 조항 번호와 요약된 내용을 포함하여 답변을 제공하세요.
        
        추가 규칙:
        답변은 간결하고 명료하게 작성하되, 필요한 경우 관련 조항의 전문을 추가적으로 인용하세요.
        세법 용어를 사용자 친화적으로 설명하여 비전문가도 쉽게 이해할 수 있도록 하세요.
        질문을 완전히 이해하기 어렵거나 모호할 경우, 사용자가 구체적으로 질문을 다시 작성할 수 있도록 유도하는 후속 질문을 하세요.
        
        답변 할 때, 개행문자 두개 이상 연속으로 절대 사용하지 마세요.

	답변 후, 사용자에게 필요할 것 같은 정보를 바탕으로 두 가지 후속 질문을 제안하세요. 각 질문의 앞뒤에 한 줄씩 띄어쓰기를 하세요. 이 질문은 원래 주제와 관련된 내용이어야 합니다.
	특정 법률 조항이나 제도가 언급될 경우, 근거가 되는 세법 조문, 시행령, 또는 관련 자료를 명시합니다.
        모든 답변은 사용자에게 법적 조언이 아닌 정보 제공 목적으로 작성된 것임을 명확히 합니다. 
        
	{context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser


In [18]:
# chain.invoke("개별소비세법이 뭐야?")
# chain.invoke("개별소비세법이 제1조가 뭐야")
# chain.invoke("교통_에너지_환경세법 뭐야?")
chain.invoke("연봉 6000에 근로소득을 가지고 있는 근로자인데 연말정산하는 방법 알려줘")

'연봉 6,000만원의 근로소득을 가지고 있는 근로자가 연말정산을 하는 방법에 대해 설명드리겠습니다. 연말정산은 매년 1월에 진행되며, 근로소득자가 한 해 동안 납부한 세금과 실제로 납부해야 할 세금의 차이를 정리하는 과정입니다. 아래는 연말정산의 주요 단계입니다.\n\n1. **소득 확인 및 공제 항목 준비**: \n   - 연봉 외에 추가적인 소득이 있는지 확인합니다. 예를 들어, 이자소득이나 배당소득이 있을 수 있습니다.\n   - 기본공제, 추가공제 항목을 준비합니다. 예를 들어, 본인 및 부양가족에 대한 기본공제, 장애인 공제, 경로우대 공제 등이 있습니다.\n   - 보험료, 의료비, 교육비, 기부금 등 특별소득공제를 받을 수 있는 지출 내역을 준비합니다.\n\n2. **소득금액 및 과세표준 계산**:\n   - 총급여(연봉)에서 근로소득공제를 차감하여 근로소득금액을 계산합니다.\n   - 근로소득금액에서 기본공제, 추가공제, 특별소득공제를 차감하여 과세표준을 계산합니다.\n\n3. **세액 계산 및 공제**:\n   - 과세표준에 따라 기본세율(6~45%)을 적용한 산출세액을 계산합니다.\n   - 산출세액에서 자녀세액공제, 연금계좌세액공제 등 세액공제를 적용하여 결정세액을 계산합니다.\n\n4. **차감납부세액 결정**:\n   - 결정세액에서 기납부세액(한 해 동안 원천징수된 세금)을 차감하여 최종 납부해야 할 세액 또는 환급받을 세액을 결정합니다. \n   - 만약 기납부세액이 결정세액보다 많다면 환급받게 되며, 반대의 경우 추가 납부해야 합니다.\n\n5. **제출 및 환급/납부**:\n   - 연말정산 서류를 회사에 제출합니다.\n   - 회사에서 국세청에 신고 후 환급 또는 추가 납부가 이루어집니다.\n\n**주의사항**:\n- 모든 공제 항목은 증빙서류가 필요하며, 국세청 홈택스를 통해 전자증빙을 발급받을 수 있습니다.\n- 공제 항목 및 세율은 매년 변경될 수 있으므로 최신 정보를 확인해야 합니다.\n\n후속 질문을 생각해보세요:\n

## 평가

In [ ]:
# LangChain 모델 래핑
langchain_model = LangchainLLMWrapper(model)

# 테스트 데이터 준비 (예시)
test_data = [
    # {
    #     "question": "개별소비세법의 목적은 무엇인가요?",
    #     "answer": chain.invoke("개별소비세법의 목적은 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("개별소비세법의 목적은 무엇인가요?")],
    #     "ground_truths": ["개별소비세법의 목적은 특정 물품과 특정 장소에 대한 소비세를 부과하여 국가 재정수입을 확보하고 소비를 조정하는 것입니다."],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("개별소비세법의 목적은 무엇인가요?")])
    # },
    {
        "question": "개별소비세법 제1조가 무엇인가요?",
        "answer": chain.invoke("개별소비세법 제1조가 무엇인가요?"),
        "contexts": [doc.page_content for doc in retriever.get_relevant_documents("개별소비세법 제1조가 무엇인가요?")],
        "ground_truths": ["개별소비세는 특정한 물품, 특정한 장소 입장행위(入場行爲), 특정한 장소에서의 유흥음식행위(遊興飮食行爲) 및 특정한 장소에서의 영업행위에 대하여 부과한다."],
        "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("개별소비세법 제1조가 무엇인가요?")])
    },
    # {
    #     "question": "조세범 처벌절차법의 목적이 무엇인가요?",
    #     "answer": chain.invoke("조세범 처벌절차법의 목적이 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("조세범 처벌절차법의 목적이 무엇인가요?")],
    #     "ground_truths": ["제1조(목적) 이 법은 조세범칙사건(犯則事件)을 공정하고 효율적으로 처리하기 위하여 조세범칙사건의 조사 및 그 처분에 관한 사항을 정함을 목적으로 한다"],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("조세범 처벌절차법의 목적이 무엇인가요?")])
    # },
    
]

# Dataset 생성
dataset = Dataset.from_list(test_data)

# 평가 실행
result = evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
    ],
    llm=langchain_model,
)

# 결과 출력
print(result)


Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

{'faithfulness': 0.3000, 'answer_relevancy': 0.8789, 'context_precision': 1.0000, 'context_recall': 1.0000}


In [1]:
from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

## GPT-4o 모델을 사용하여 평가 
model_name = "gpt-4o"
model = ChatOpenAI(model=model_name)
langchain_model = LangchainLLMWrapper(model)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)
metrics = [
    LLMContextRecall(llm=langchain_model),
    LLMContextPrecisionWithReference(llm=langchain_model),
    Faithfulness(llm=langchain_model),
    AnswerRelevancy(llm=langchain_model, embeddings=eval_embedding)
]
result = evaluate(dataset=dataset, metrics=metrics)

NameError: name 'ChatOpenAI' is not defined

In [ ]:
result

In [ ]:
# DataFrame 생성
df_result = pd.DataFrame(test_data)

# 최대 열 너비 설정
pd.set_option('display.max_colwidth', None)

# 최대 행 수 설정
pd.set_option('display.max_rows', None)

df_result

,question,answer,contexts,ground_truths,reference
0,개별소비세법의 목적은 무엇인가요?,"개별소비세법의 목적은 특정 물품이나 서비스에 대해 소비세를 부과하여 세수 확보와 소비 패턴 조정을 통해 공공재원을 조달하고 사회적 비용을 반영하는 것입니다. 이를 통해 사치품이나 환경에 미치는 영향이 큰 제품의 소비를 억제하고, 공공 목적으로 사용되는 자금을 확보하는 데 기여합니다. \n\n추가적으로 궁금하신 사항이 있나요?\n\n1. 개별소비세가 부과되는 구체적인 품목에는 어떤 것들이 있는지 알고 싶으신가요?\n \n2. 개별소비세의 세율은 어떻게 결정되는지 궁금하신가요?","[개별소비세법 시행규칙\n[시행 2024. 3. 22.] [기획재정부령 제1047호, 2024. 3. 22., 일부개정]\n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336, 제4조(과세시기) 개별소비세는 다음 각 호에 따른 반출, 수입신고, 입장, 유흥음식행위 또는 영업행위를 할 때에 그 행위\n당시의 법령에 따라 부과한다. 다만, 제3조제4호의 경우에는 「관세법」에 따른다. <개정 2015. 12. 15.>\n1. 물품에 대한 개별소비세: 과세물품을 제조장에서 반출할 때 또는 수입신고를 할 때\n2. 입장행위에 대한 개별소비세: 과세장소에 입장할 때\n3. 유흥음식행위에 대한 개별소비세: 유흥음식행위를 할 때\n 4. 영업행위에 대한 개별소비세: 과세영업장소의 영업행위를 할 때\n[전문개정 2010. 1. 1.], 제1조(목적) 이 법은 주세의 과세 요건 및 절차를 규정함으로써 주세를 공정하게 과세하고, 납세의무의 적정한 이행을\n확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다., 제1조(목적) 이 규칙은 「외국인관광객 등에 대한 부가가치세 및 개별소비세 특례규정」에서 위임된 사항과 그 시행에\n필요한 사항을 규정함을 목적으로 한다.\n[전문개정 2010. 4. 13.], 제1조(목적) 이 법은 조세(租稅)의 감면 또는 중과(重課) 등 조세특례와 이의 제한에 관한 사항을 규정하여 과세(課稅)의\n공평을 도모하고 조세정책을 효율적으로 수행함으로써 국민경제의 건전한 발전에 이바지함을 목적으로 한다. <개정\n2020. 6. 9.>\n[전문개정 2010. 1. 1.]]",[개별소비세법의 목적은 특정 물품과 특정 장소에 대한 소비세를 부과하여 국가 재정수입을 확보하고 소비를 조정하는 것입니다.],"개별소비세법 시행규칙\n[시행 2024. 3. 22.] [기획재정부령 제1047호, 2024. 3. 22., 일부개정]\n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336\n제4조(과세시기) 개별소비세는 다음 각 호에 따른 반출, 수입신고, 입장, 유흥음식행위 또는 영업행위를 할 때에 그 행위\n당시의 법령에 따라 부과한다. 다만, 제3조제4호의 경우에는 「관세법」에 따른다. <개정 2015. 12. 15.>\n1. 물품에 대한 개별소비세: 과세물품을 제조장에서 반출할 때 또는 수입신고를 할 때\n2. 입장행위에 대한 개별소비세: 과세장소에 입장할 때\n3. 유흥음식행위에 대한 개별소비세: 유흥음식행위를 할 때\n 4. 영업행위에 대한 개별소비세: 과세영업장소의 영업행위를 할 때\n[전문개정 2010. 1. 1.]\n제1조(목적) 이 법은 주세의 과세 요건 및 절차를 규정함으로써 주세를 공정하게 과세하고, 납세의무의 적정한 이행을\n확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n제1조(목적) 이 규칙은 「외국인관광객 등에 대한 부가가치세 및 개별소비세 특례규정」에서 위임된 사항과 그 시행에\n필요한 사항을 규정함을 목적으로 한다.\n[전문개정 2010. 4. 13.]\n제1조(목적) 이 법은 조세(租稅)의 감면 또는 중과(重課) 등 조세특례와 이의 제한에 관한 사항을 규정하여 과세(課稅)의\n공평을 도모하고 조세정책을 효율적으로 수행함으로써 국민경제의 건전한 발전에 이바지함을 목적으로 한다. <개정\n2020. 6. 9.>\n[전문개정 2010. 1. 1.]"
1,조세범 처벌절차법의 목적이 무엇인가요?,조세범 처벌절차법의 목적은 세법을 위반한 자에 대한 형벌에 관한 사항을 규정하여 세법의 실효성을 높이고 국민의 건전한 납세 의식을 확립하는 데 있습니다. 이 법은 세법 집행의 공정성과 세금 납부의 적정성을 보장하기 위한 제도적 장치를 마련하고 있습니다.\n\n세법과 관련된 다른 질문이 있으신가요? 예를 들어:\n\n1. 조세범 처벌절차법에 의해 처벌받는 구체적인 행위는 어떤 것이 있는지 알고 싶으신가요?\n\n2. 조세범 처벌절차법과 관련된 최신 개정 사항이 궁금하신가요?,"[조세범 처벌절차법\n[시행 2023. 1. 17.] [법률 제19212호, 2023. 1. 17., 일부개정]\n기획재정부 (조세법령운용팀) 044-215-4151\n 제1장 총칙, 제1조(목적) 이 영은 「조세범 처벌절차법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다., 제1조(목적) 이 법은 주세의 과세 요건 및 절차를 규정함으로써 주세를 공정하게 과세하고, 납세의무의 적정한 이행을\n확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다., 제1조(목적) 이 법은 세법을 위반한 자에 대한 형벌에 관한 사항을 규정하여 세법의 실효성을 높이고 국민의 건전한 납\n세의식을 확립함을 목적으로 한다. <개정 2018. 12. 31.>, 제39조(벌칙) 재평가세에 관한 범칙행위에 대하여는 조세범처벌법을 적용한다.]",[제1조(목적) 이 법은 조세범칙사건(犯則事件)을 공정하고 효율적으로 처리하기 위하여 조세범칙사건의 조사 및 그 처분에 관한 사항을 정함을 목적으로 한다],"조세범 처벌절차법\n[시행 2023. 1. 17.] [법률 제19212호, 2023. 1. 17., 일부개정]\n기획재정부 (조세법령운용팀) 044-215-4151\n 제1장 총칙\n제1조(목적) 이 영은 「조세범 처벌절차법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n제1조(목적) 이 법은 주세의 과세 요건 및 절차를 규정함으로써 주세를 공정하게 과세하고, 납세의무의 적정한 이행을\n확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n제1조(목적) 이 법은 세법을 위반한 자에 대한 형벌에 관한 사항을 규정하여 세법의 실효성을 높이고 국민의 건전한 납\n세의식을 확립함을 목적으로 한다. <개정 2018. 12. 31.>\n제39조(벌칙) 재평가세에 관한 범칙행위에 대하여는 조세범처벌법을 적용한다."
